<a href="https://colab.research.google.com/github/i-am-U-hyUn/Anti-Spoofing/blob/main/detect_face_spoofing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnxruntime --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision opencv-python insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s

In [ ]:
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import insightface
from insightface.app import FaceAnalysis
import os
import sys
from typing import Type, Any, Callable, Union, List, Optional, Tuple


# ResNet 모델 구현
class BasicBlock(torch.nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[torch.nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = torch.nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = torch.nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = norm_layer(planes)
        self.relu = torch.nn.ReLU(inplace=True)
        self.conv2 = torch.nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(torch.nn.Module):
    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[torch.nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., torch.nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = torch.nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = torch.nn.Conv2d(inplanes, width, kernel_size=1, stride=1, bias=False)
        self.bn1 = norm_layer(width)
        self.conv2 = torch.nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = torch.nn.Conv2d(width, planes * self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = torch.nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(torch.nn.Module):
    def __init__(
        self,
        block,
        layers,
        num_classes=1000,
        zero_init_residual=False,
        groups=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
        fp16=False
    ):
        super(ResNet, self).__init__()
        self.fp16 = fp16
        if norm_layer is None:
            norm_layer = torch.nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = torch.nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = torch.nn.ReLU(inplace=True)
        self.maxpool = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.fc = torch.nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = torch.nn.Sequential(
                torch.nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return torch.nn.Sequential(*layers)

    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        features = torch.flatten(x, 1)

        logits = self.fc(features)
        return (features, logits)

    def forward(self, x):
        return self._forward_impl(x)


def create_resnet50(num_classes=2):
    """자체 ResNet50 모델 생성 함수"""
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes)


# 표준 PyTorch ResNet50 생성 함수
def create_torchvision_resnet50(num_classes=2):
    """표준 torchvision ResNet50 모델 생성 함수"""
    from torchvision.models import resnet50
    model = resnet50(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    return model


# 모델 로드 함수
def load_model(model_path, model_type="resnet50", num_classes=2):
    if model_type == "resnet50":
        model = create_resnet50(num_classes=num_classes)
    elif model_type == "swin_v2_b":
        from torchvision.models import swin_v2_b
        model = swin_v2_b(pretrained=False)
        model.head = torch.nn.Linear(model.head.in_features, num_classes)
    elif model_type == "mobilenet_v3_small":
        from torchvision.models import mobilenet_v3_small
        model = mobilenet_v3_small(pretrained=False)
        model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, num_classes)
    else:
        print(f"Unsupported model type: {model_type}")
        sys.exit(1)

    # 체크포인트 불러오기
    try:
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

        # 체크포인트 형식 확인 및 모델 state_dict 추출
        if isinstance(checkpoint, dict):
            if 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
                print("체크포인트에서 'state_dict' 키를 불러왔습니다.")
            elif 'state_dict_ema' in checkpoint:
                state_dict = checkpoint['state_dict_ema']
                print("체크포인트에서 'state_dict_ema' 키를 불러왔습니다.")
            else:
                state_dict = checkpoint
                print("체크포인트 전체를 state_dict로 사용합니다.")
        else:
            state_dict = checkpoint
            print("체크포인트를 직접 state_dict로 사용합니다.")

        # 'module.' 접두사 제거 (분산 학습의 경우)
        new_state_dict = {}
        for k, v in state_dict.items():
            name = k
            if k.startswith('module.'):
                name = k[7:]  # module. 제거
            new_state_dict[name] = v

        # 모델에 가중치 로드 시도 (strict=False로 일부 누락된 키 허용)
        model.load_state_dict(new_state_dict, strict=False)
        print("모델 가중치 로드 성공 (일부 가중치는 누락될 수 있음)")

    except Exception as e:
        print(f"모델 로딩 중 오류 발생: {e}")
        print("ImageNet 사전 훈련된 모델로 대체...")

        # 표준 torchvision ResNet50 모델로 대체
        model = create_torchvision_resnet50(num_classes)
        print("ImageNet 사전 훈련된 ResNet50 모델로 대체되었습니다.")

    model.eval()
    return model


# 얼굴 검출기 초기화
def init_face_detector():
    face_detector = FaceAnalysis(providers=['CPUExecutionProvider'])
    face_detector.prepare(ctx_id=0, det_size=(640, 640))
    return face_detector


# 이미지 전처리
def preprocess_image(image, face_only=False, face_detector=None, input_size=224):
    h, w = image.shape[:2]

    if face_only and face_detector is not None:
        # 얼굴 검출
        faces = face_detector.get(image)
        if faces and len(faces) > 0:
            # 얼굴을 찾았을 때
            face = faces[0]
            bbox = face.bbox.astype(int)
            # 20픽셀 확장
            x1 = max(0, bbox[0] - 20)
            y1 = max(0, bbox[1] - 20)
            x2 = min(w, bbox[2] + 20)
            y2 = min(h, bbox[3] + 20)
            face_img = image[y1:y2, x1:x2]
            if face_img.size == 0:
                # 얼굴 영역이 없는 경우 중앙 크롭
                return None, None, None
            processed_img = face_img
            face_bbox = (x1, y1, x2, y2)
        else:
            # 얼굴이 감지되지 않으면 중앙 크롭
            if w >= 700 and h >= 700:
                size = 500
                center_x, center_y = w // 2, h // 2
                x1 = max(0, center_x - size // 2)
                y1 = max(0, center_y - size // 2)
                x2 = min(w, center_x + size // 2)
                y2 = min(h, center_y + size // 2)
                processed_img = image[y1:y2, x1:x2]
                face_bbox = (x1, y1, x2, y2)
            else:
                processed_img = image
                face_bbox = (0, 0, w, h)
    else:
        # 전체 이미지 사용
        processed_img = image
        face_bbox = (0, 0, w, h)

    # 이미지 변환
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    tensor_img = transform(processed_img).unsqueeze(0)
    return tensor_img, processed_img, face_bbox


# 추론 실행
def run_inference(model, image, model_type="resnet50"):
    with torch.no_grad():
        try:
            if model_type.startswith("mobilenet") or model_type.startswith("shufflenet"):
                outputs = model(image)
            else:
                # ResNet 및 Swin 모델의 경우
                try:
                    features, outputs = model(image)
                except:
                    outputs = model(image)

            probabilities = F.softmax(outputs, dim=1)
            return probabilities.numpy()[0]
        except Exception as e:
            print(f"추론 중 오류 발생: {e}")
            # 오류 발생 시 기본값 반환 (50% 확률)
            return np.array([0.5, 0.5])


def main():
    # 설정
    model_path = "full_resnet50.pth"  # 모델 경로 지정
    model_type = "resnet50"  # 모델 유형에 따라 설정
    face_only = False  # face_swin_v2_base.pth 사용 시 True로 설정
    input_size = 224
    image_path = "1_dpi.png"  # 현재 경로의 이미지 파일

    # 모델 및 얼굴 검출기 로드
    model = load_model(model_path, model_type, num_classes=2)
    face_detector = init_face_detector() if face_only else None

    print(f"모델 로딩 완료: {model_type}")

    # 이미지 불러오기
    if not os.path.exists(image_path):
        print(f"오류: 이미지 파일을 찾을 수 없습니다: {image_path}")
        return

    frame = cv2.imread(image_path)
    if frame is None:
        print(f"오류: 이미지를 읽을 수 없습니다: {image_path}")
        return

    # 원본 이미지 복사 (결과 표시용)
    result_image = frame.copy()

    # 이미지 전처리
    tensor_img, processed_img, face_bbox = preprocess_image(
        frame, face_only, face_detector, input_size)

    if tensor_img is None:
        print("얼굴을 감지할 수 없습니다.")
        cv2.putText(result_image, "No face detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Face Anti-Spoofing Result', result_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return

    # 추론
    probs = run_inference(model, tensor_img, model_type)

    # 결과 표시 (README에 따르면 spoof=1, live=0)
    spoof_prob = probs[1]  # 스푸핑 확률
    is_spoof = spoof_prob > 0.5
    prob_text = f"Spoof: {spoof_prob:.4f}, Live: {probs[0]:.4f}"
    result_text = "SPOOF (위조)" if is_spoof else "LIVE (실제)"
    color = (0, 0, 255) if is_spoof else (0, 255, 0)

    # 결과 이미지에 표시
    cv2.putText(result_image, result_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
    cv2.putText(result_image, prob_text, (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # 얼굴 영역 표시
    if face_bbox is not None:
        x1, y1, x2, y2 = face_bbox
        cv2.rectangle(result_image, (x1, y1), (x2, y2), color, 3)

    # 처리된 얼굴 이미지 표시 (선택 사항)
    if processed_img is not None and face_only:
        h, w = processed_img.shape[:2]
        h_result, w_result = result_image.shape[:2]

        # 크기 조정 (결과 이미지의 1/4 크기로)
        display_w = w_result // 4
        display_h = int(h * display_w / w)
        display_img = cv2.resize(processed_img, (display_w, display_h))

        # 오른쪽 상단에 작은 이미지로 표시
        h_display, w_display = display_img.shape[:2]
        result_image[10:10+h_display, w_result-10-w_display:w_result-10] = display_img

    # 최종 결과 출력
    print(f"분석 결과: {result_text}")
    print(f"확률: {prob_text}")

    # 결과 이미지 표시
    # cv2.imshow('Face Anti-Spoofing Result', result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # 결과 이미지 저장 (선택 사항)
    result_file = 'result1_dpi.jpg'
    cv2.imwrite(result_file, result_image)
    print(f"결과 이미지가 저장되었습니다: {result_file}")


if __name__ == "__main__":
    main()

체크포인트에서 'state_dict' 키를 불러왔습니다.
모델 가중치 로드 성공 (일부 가중치는 누락될 수 있음)
모델 로딩 완료: resnet50
분석 결과: LIVE (실제)
확률: Spoof: 0.0755, Live: 0.9245
결과 이미지가 저장되었습니다: result1_dpi.jpg


In [ ]:
from PIL import Image

# 이미지 열기
img = Image.open("1.png")

# 원본 사이즈 확인
original_size = img.size  # (2800, 2126)

# 절반 크기로 리사이즈
new_size = (original_size[0] // 2, original_size[1] // 2)
resized_img = img.resize(new_size, Image.LANCZOS)

# 저장
resized_img.save("1_resized.png")

In [ ]:
from PIL import Image

# 원본 이미지 열기
img = Image.open("1.png")
original_dpi = img.info.get("dpi")

print(f"Original DPI: {original_dpi}")

# 절반 DPI로 설정
new_dpi = (original_dpi[0] // 2, original_dpi[1] // 2)
print(f"New DPI: {new_dpi}")

# 저장
img.save("1_dpi.png", dpi=new_dpi)

Original DPI: (143.99259999999998, 143.99259999999998)
New DPI: (71.0, 71.0)
